In [1]:
from astropy.table import Table, vstack
import sys
sys.path.append('/Users/fardila/Documents/GitHub/HSC_vs_hydro/')
from functions import *

Created TAP+ (v1.0) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


      +------------------------------------------------------------+
      |             Space Telescope Tables Package                 |
      |                  TABLES Version 3.18.3                     |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      +------------------------------------------------------------+
tables/:
 fitsio/        tbplot/         tobsolete/      ttools/


      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |
  

9/12/18: started modularizing the function more. still working on `get_isos`, `get_extrapolated_masses`, `get_2d_masses`

9/13/18: need to figure out how to restructure output array in `get_masses`

9/13/18: I think I figured out the restructuring. Though I am still not sure whether numpy array is better than astropy table (maybe not...I lean towards making a table). Need to add to script files and run on full sample.

In [87]:
#new functions

def get_iso(sim_file, sim_name, resolution, intMode='mean', components='cen', gal_n=0):
    
    '''
    get iso using galSBP
    '''

    # Load maps
    mass_map_cen, mass_map_cen_icl, pixel_scale, m_cat = get_mass_maps(sim_file, gal_n=gal_n)

    #ouput maps
    maps_location='/Users/fardila/Documents/GitHub/HSC_vs_hydro/Figures/fits_files/{0}/'.format(resolution)

    file_name=sim_name+'_'+str(gal_n)+'_xy'
    fits_prefix = maps_location + file_name

    #components
    if components == 'cen':
        save_to_fits(mass_map_cen, fits_prefix + '_cen.fits')
        data=mass_map_cen
    elif components == 'cen+icl':
        save_to_fits(mass_map_cen_icl, fits_prefix + '_cen+icl.fits')
        data=mass_map_cen_icl
    else:
        raise ValueError('only cen or cen+icl allowed for now')


    suffix='_'+components

    #central pixels
    x0=len(data)/2.
    y0=len(data)/2.

    ###########################################################################
    #ellipse information
    #########################
    if resolution == 'highres':
        #get ellipse information from iso file of quick
        iso_quick = load_pkl('/Users/fardila/Documents/GitHub/HSC_vs_hydro/Figures/fits_files/quick/{0}_{1}_xy_cen_ellip_3.pkl'.format(sim_name, gal_n))
        q = 1- iso_quick['ell'][-1]
        theta = iso_quick['pa'][-1]* np.pi /180.
    elif resolution == 'quick':
        #get background
        bkg = sep.Background(data, bw=10, bh=10, fw=5, fh=5)
        bkg_subtraced_data = data - bkg

        thresh = 50 * bkg.globalrms
        objects = sep.extract(bkg_subtraced_data, thresh, minarea = 100,
                              deblend_nthresh=24, deblend_cont=0.1)

        #find object closest to image center
        obj = find_closest(objects, x0=x0, y0=y0)

        #ellipse parameters
        theta = obj['theta']
        q = obj['b']/ obj['a']

    ###########################################################################
    #1D masses from galSBP
    iso, iso_bin = galSBP.galSBP(maps_location+file_name+suffix+'.fits',
                                     galX=x0,
                                     galY=y0,
                                     galQ=q,
                                     galPA=theta* 180. / np.pi,
                                     maxSma=250,
                                     iniSma=50.0,
                                     stage=3,
                                     intMode=intMode,
                                     ellipStep=0.05,
                                     pix=pixel_scale,
                                     zpPhoto=0.0,
                                     isophote=x_isophote,
                                     xttools=x_ttools,
                                     recenter=True,
                                     savePng=False,
                                     verbose=True,
                                     uppClip=3.0,
                                     lowClip=3.0,
                                     nClip=2)


    ###########################################################################
    iso['sma_kpc'] = iso['sma'] * pixel_scale
    iso['intens_kpc']=iso['intens'] / (pixel_scale**2)

    return iso
##################################################

def get_1d_masses(iso):
    '''
    measures 1d mass from iso at various radii
    '''
    
    m_1d_10, m_1d_30, m_1d_100, m_1d_500, m_1d_800 = oneD_mass(iso, 10.),\
                                            oneD_mass(iso, 30.),\
                                            oneD_mass(iso, 100.),\
                                            oneD_mass(iso, 500.),\
                                            oneD_mass(iso, 800.)

    oneD_masses = np.array([(m_1d_10, m_1d_30, m_1d_100, m_1d_500, m_1d_800),],
                           dtype=[('m_1d_10',np.float),('m_1d_30',np.float),('m_1d_100',np.float),('m_1d_500',np.float),
                                  ('m_1d_800',np.float)])
    return oneD_masses

def get_2d_masses(sim_file, sim_name, resolution, gal_n=0):
    '''
    measures 2d mass from mass map at various radii
    '''
   
    #central pixels
    x0=150.
    y0=150.

    # Load maps
    mass_map_cen, mass_map_cen_icl, pixel_scale, m_cat = get_mass_maps(sim_file, gal_n=gal_n)

    #postage mass
    m_post = np.log10(np.sum(mass_map_cen))
    m_post_icl = np.log10(np.sum(mass_map_cen_icl))


    #ouput maps
    maps_location='/Users/fardila/Documents/GitHub/HSC_vs_hydro/Figures/fits_files/quick_800/'

    file_name=sim_name+'_'+str(gal_n)+'_xy'
    fits_prefix = maps_location + file_name
    #save_to_fits(mass_map_cen, fits_prefix + '_cen.fits')
    # save_to_fits(img_cen_sat, fits_prefix + '_cen_sat.fits')
    # save_to_fits(img_cen_icl, fits_prefix + '_cen_icl.fits')
    # save_to_fits(img_all, fits_prefix + '_all.fits')

    data=mass_map_cen
    suffix='_cen'

    ###########################################################################
    ###########################################################################
    #ellipse information
    #########################
    if resolution == 'highres':
        #get ellipse information from iso file of quick
        iso_quick = load_pkl('/Users/fardila/Documents/GitHub/HSC_vs_hydro/Figures/fits_files/quick/{0}_{1}_xy_cen_ellip_3.pkl'.format(sim_name, gal_n))
        q = 1- iso_quick['ell'][-1]
        theta = iso_quick['pa'][-1]* np.pi /180.
    elif resolution == 'quick':
        #get background
        bkg = sep.Background(data, bw=10, bh=10, fw=5, fh=5)
        bkg_subtraced_data = data - bkg

        thresh = 50 * bkg.globalrms
        objects = sep.extract(bkg_subtraced_data, thresh, minarea = 100,
                              deblend_nthresh=24, deblend_cont=0.1)

        #find object closest to image center
        obj = find_closest(objects, x0=x0, y0=y0)

        #ellipse parameters
        theta = obj['theta']
        q = obj['b']/ obj['a']

    a_10, a_30, a_100, a_500, a_800 = (10. / pixel_scale), (30. / pixel_scale), (100. / pixel_scale),\
                                        (500. / pixel_scale), (800. / pixel_scale)
    b_10, b_30, b_100, b_500, b_800 =  a_10 * q, a_30 * q, a_100 * q, a_500 * q, a_800 * q
    
    #2D masses
    flux_10, fluxerr_10, flag_10 = sep.sum_ellipse(data, x0, y0,
                                                   a_10, b_10, theta)
    flux_30, fluxerr_30, flag_30 = sep.sum_ellipse(data, x0, y0,
                                                   a_30, b_30, theta)
    flux_100, fluxerr_100, flag_100 = sep.sum_ellipse(data, x0, y0,
                                                      a_100, b_100, theta)
    flux_500, fluxerr_500, flag_500 = sep.sum_ellipse(data, x0, y0,
                                                      a_500, b_500, theta)
    flux_800, fluxerr_800, flag_800 = sep.sum_ellipse(data, x0, y0,
                                                      a_800, b_800, theta)
    
    m_2d_10, m_2d_30, m_2d_100, m_2d_500, m_2d_800 = np.log10(flux_10), \
                                                        np.log10(flux_30), \
                                                        np.log10(flux_100), \
                                                        np.log10(flux_500), \
                                                        np.log10(flux_800)




    # plot background-subtracted image
    m, s = np.mean(data), np.std(data)
    fig, ax = plt.subplots()
    im = ax.imshow(data, interpolation='nearest', cmap=plt.get_cmap('viridis'),
                   vmin=m-s, vmax=m+s, origin='lower')

    # plot an ellipse for each object
    e_30 = Ellipse(xy=(x0, y0),
                 width=a_30*2,
                 height=b_30*2,
                 angle=theta * 180. / np.pi)
    e_30.set_facecolor('none')
    e_30.set_edgecolor('red')
    ax.add_artist(e_30)

    e_100 = Ellipse(xy=(x0, y0),
                 width=a_100*2,
                 height=b_100*2,
                 angle=theta * 180. / np.pi)
    e_100.set_facecolor('none')
    e_100.set_edgecolor('red')
    ax.add_artist(e_100)


    plt.savefig('/Users/fardila/Documents/GitHub/HSC_vs_hydro/Figures/ellipses/{0}/{1}'.format(resolution, file_name))
    plt.clf()

    ###########################################################################
   
    
    twoD_masses = np.array([(m_2d_10,m_2d_30, m_2d_100, m_2d_500, m_2d_800),],
                           dtype=[('m_2d_10','f8'),('m_2d_30','f8'),('m_2d_100','f8'),('m_2d_500','f8'),('m_2d_800','f8')])
    return twoD_masses

def get_extrapolated_masses(iso, rs):
    '''
    measures integrated masses from extrapolation of iso at various radii
    '''
                           
    masses = tuple([extrapolated_1D_mass(iso, r) for r in rs])
    dtypes = [('extrapolated_m_{0}'.format(r),'<f8') for r in rs]

    extrapolated_masses = np.array([masses,], dtype=dtypes)
    
    return extrapolated_masses

def get_cat_post_masses(sim_file, gal_n):
    '''
    measures catalog and postage masses from hdf5 file
    '''
    # Load maps
    mass_map_cen, mass_map_cen_icl, pixel_scale, m_cat = get_mass_maps(sim_file, gal_n=gal_n)

    #postage mass
    m_post = np.log10(np.sum(mass_map_cen))
    m_post_icl = np.log10(np.sum(mass_map_cen_icl))
    
    cat_post_masses = np.array([(m_cat, m_post, m_post_icl),], dtype=[('m_cat','f8'),('m_post','f8'),('m_post_icl','f8')])
    return cat_post_masses


def get_masses(iso, sim_file, sim_name, resolution, rs, gal_n=0):
    '''
    get all masses and save as single array
    '''
    
    masses_1d = get_1d_masses(iso)
    masses_2d = get_2d_masses(sim_file, sim_name, resolution, gal_n=0)
    masses_extrapolated = get_extrapolated_masses(iso, rs)
    masses_cat_post = get_cat_post_masses(sim_file, gal_n)
    
    masses_array = (masses_1d, masses_2d, masses_extrapolated, masses_cat_post)
    
    #merges structured arrays and flatten, while preserving structured form with keywords.
    masses = np.lib.recfunctions.merge_arrays(masses_array, flatten = True, usemask = False)
    
    return Table(masses)

In [46]:
masses[0]['m_cat']

12.449948294587058

In [49]:
get_masses(iso, TNG_file_quick,'TNG', resolution, rs, gal_n=0)['m_cat']

12.449948294587058

<Figure size 432x288 with 0 Axes>

In [3]:
TNG_file_quick = '/Users/fardila/Documents/GitHub/HSC_vs_hydro/Data/TNG/galaxies_stellarmaps_tng75_11.2.hdf5'

TNG_file_highres = '/Users/fardila/Documents/GitHub/HSC_vs_hydro/Data/TNG/galaxies_stellarmaps_tng75_11.2_highres.hdf5'


In [83]:
resolution = 'quick'
i = 0
rs=[100,300,500]

In [88]:
isos_tng = []
masses_tng = []

for i in range(5):
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print('^^^^^^^^GALAXY '+str(i)+'^^^^^^^^^^^^^^')
    print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    
    iso = get_iso(TNG_file_quick,'TNG', resolution, intMode='mean', components='cen', gal_n=i)
    masses = get_masses(iso, TNG_file_quick,'TNG', resolution, rs, gal_n=0)

    isos_tng.append(iso)
    masses_tng.append(masses)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^GALAXY 0^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
----------------------------------------------------------------------------------------------------
###      galX, galY :  150.0 150.0
###      galR :  20.0
###      iniSma, maxSma :  50.0 250
###      Stage :  3
###      Step :  0.05
----------------------------------------------------------------------------------------------------
##       Set up the Ellipse configuration
----------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
##       Start the Ellipse Run: Attempt  1
----------------------------------------------------------------------------------------------------
###      Origin Image  : /Users/fardila/Documents/GitHub/HSC_vs_hydro/Figures/fits_files/quick/TNG_0_xy_cen.fits
###      Input Image   : temp_UP3N6.fits
###      Output Binary : /U

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [89]:
outfile_loc = '/Users/fardila/Documents/GitHub/HSC_vs_hydro/Data/TNG/'
pkl_masses = open(outfile_loc+'TNG_masses_{0}_TEST.pkl'.format(resolution),'wb')
pickle.dump(vstack(masses_tng),pkl_masses)
pkl_masses.close()

pkl_isos = open(outfile_loc+'TNG_isos_{0}_TEST.pkl'.format(resolution),'wb')
pickle.dump(isos_tng,pkl_isos)
pkl_isos.close()

In [105]:
resolution = 'quick'

In [106]:
#loading isos and masses
outfile_loc = '/Users/fardila/Documents/GitHub/HSC_vs_hydro/Data/Illustris/'
loaded_masses = load_pkl(outfile_loc+'Illustris_masses_{0}_TEST.pkl'.format(resolution))
loaded_isos = load_pkl(outfile_loc+'Illustris_isos_{0}_TEST.pkl'.format(resolution))

In [107]:
loaded_masses

m_1d_10,m_1d_30,m_1d_100,m_1d_500,m_1d_800,m_2d_10,m_2d_30,m_2d_100,m_2d_500,m_2d_800,extrapolated_m_300,extrapolated_m_500,extrapolated_m_800,m_cat,m_post,m_post_icl
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
12.07446572383834,12.26298276326777,12.374762800668066,12.40512898491162,12.40512898491162,11.59211819401873,11.980390530978925,12.211540578547103,12.383923709073407,12.40734361924449,12.542989892942217,12.61766386948733,12.68486401281356,12.414851125832998,12.413413089563061,12.414139258776743
11.927018793377632,12.04968407599899,12.15002381623849,12.204892899845495,12.204892899845495,11.59211819401873,11.980390530978925,12.211540578547103,12.383923709073407,12.40734361924449,12.27071152882789,12.326469104376475,12.377603086139704,12.222039953054319,12.210375448739363,12.210939215934795
12.033377929874863,12.210542351117176,12.370330939909248,12.450512810738731,12.450512810738731,11.59211819401873,11.980390530978925,12.211540578547103,12.383923709073407,12.40734361924449,12.586701939148073,12.661028314458395,12.720500153039183,12.33545120600355,12.327935962728256,12.32893027010405
12.031796856449796,12.19230494291991,12.276661460062718,12.30839541497177,12.30839541497177,11.59211819401873,11.980390530978925,12.211540578547103,12.383923709073407,12.40734361924449,12.402591635257552,12.446728323001595,12.481835181810462,12.306992548357814,12.29988460509768,12.30078232870949
11.810181949855185,11.934534990028611,12.013103770652638,12.043129728781983,12.043129728781983,11.59211819401873,11.980390530978925,12.211540578547103,12.383923709073407,12.40734361924449,12.126104570884152,12.167354428842646,12.200799972021805,12.034769685894226,12.028936908777807,12.029065287160565


In [108]:
loaded_isos

[<Table length=124>
    sma        intens       int_err    ...      sma_kpc          intens_kpc  
  float64     float64       float64    ...      float64           float64    
 --------- -------------- ------------ ... ------------------ ---------------
       0.0 114307000000.0          nan ...                0.0   4018605468.75
 0.5095395  88062600000.0 3783796000.0 ...           2.717544   3095950781.25
 0.5350164  86926510000.0 3962977000.0 ... 2.8534207999999994 3056010117.1875
 0.5617672  85751530000.0 4150765000.0 ... 2.9960917333333335 3014702226.5625
 0.5898556  84537970000.0 4347539000.0 ... 3.1458965333333335 2972038007.8125
 0.6193483  83285670000.0 4554001000.0 ...  3.303190933333333 2928011835.9375
 0.6503156  81994870000.0 4770648000.0 ... 3.4683498666666663 2882632148.4375
 0.6828314  80666080000.0 4998252000.0 ... 3.6417674666666664    2835916875.0
 0.7169729  79300650000.0 5237467000.0 ... 3.8238554666666666 2787913476.5625
 0.7528216  77899050000.0 5489193000.0 ...  

# everythin seems to work! Will delete after running on all galaxies

In [123]:
outfile_loc = '/Users/fardila/Documents/GitHub/HSC_vs_hydro/Data/TNG/'
resolution = 'quick'

In [126]:
load_pkl(outfile_loc+'TNG_isos_{0}.pkl'.format(resolution))

[<Table length=125>
    sma        intens       int_err    ...      sma_kpc          intens_kpc  
  float64     float64       float64    ...      float64           float64    
 --------- -------------- ------------ ... ------------------ ---------------
       0.0 129035600000.0          nan ...                0.0    4536407812.5
 0.5095395 102718700000.0 4904136000.0 ...           2.717544  3611204296.875
 0.5350164 101464400000.0 5140003000.0 ... 2.8534207999999994    3567107812.5
 0.5617672 100154000000.0 5386708000.0 ... 2.9960917333333335    3521039062.5
 0.5898556  98784520000.0 5644844000.0 ... 3.1458965333333335   3472893281.25
 0.6193483  97354700000.0 5914769000.0 ...  3.303190933333333  3422626171.875
 0.6503156  95861570000.0 6197040000.0 ... 3.4683498666666663 3370133320.3125
 0.6828314  94303060000.0 6492211000.0 ... 3.6417674666666664  3315341953.125
 0.7169729  92677010000.0 6800725000.0 ... 3.8238554666666666 3258176132.8125
 0.7528216  90980940000.0 7123194000.0 ...  

In [125]:
len(load_pkl(outfile_loc+'TNG_isos_{0}.pkl'.format(resolution)))

235